In [1]:
def get_flights(date_1, date_2):
    """ Returns the number of flights in a date interval  """
    import json
    from dateutil.parser import parse
    flights = {
        "flights": abs((parse(date_2) - parse(date_1)).days) 
    }
    return json.dumps(flights)

def my_cat_born_date():
    """ Returns my cat's born date """
    import datetime, random
    from dateutil.relativedelta import relativedelta
    
    # Calculate the date as ten years ago  
    ten_years_ago = datetime.date.today() - relativedelta(years=10) 
    
    cat_born_date = {
        "cat_born_date": ten_years_ago.strftime("%Y-%m-%d")
    }
    return json.dumps(cat_born_date)

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_flights",            
            "description": "returns the number of flights between two dates",
            "parameters": {
                "type": "object",
                "properties": {
                    "date_1": {"type": "string", "description": "the first date"},
                    "date_2": {"type": "string", "description": "the second date"}
                }
            },
            "required": ["date_1", "date_2"]
        }
    },
    {
        "type": "function",
        "function": {
            "name": "my_cat_born_date",            
            "description": "returns my cat's born date",
            "parameters": {
                "type": "object",
                "properties": {

                }
            },
            "required": []
        }
    }
]

In [2]:
import os
from dotenv import load_dotenv # requires python-dotenv

load_dotenv("./../credentials_my.env")
MODEL = os.environ["GPT4-1106-128k"]
QUESTION = "How many flights do we have between my cat born date and Easter 2021?"

In [3]:
from openai import AzureOpenAI

In [4]:
# Create the client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview", # at least 2024-02-15-preview
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [5]:
# Create an assistant
assistant = client.beta.assistants.create(
    name="Smart Assistant",
    instructions=f"You are a helpful AI assistant who helps answering questions",
    tools = tools,
    model = MODEL
)

print(assistant.model_dump_json(indent=2))

{
  "id": "asst_RN5yZ4Kk1nzxC8Sx6EskFfxs",
  "created_at": 1710871132,
  "description": null,
  "file_ids": [],
  "instructions": "You are a helpful AI assistant who helps answering questions",
  "metadata": {},
  "model": "gpt4-1106-128k",
  "name": "Smart Assistant",
  "object": "assistant",
  "tools": [
    {
      "function": {
        "name": "get_flights",
        "description": "returns the number of flights between two dates",
        "parameters": {
          "type": "object",
          "properties": {
            "date_1": {
              "type": "string",
              "description": "the first date"
            },
            "date_2": {
              "type": "string",
              "description": "the second date"
            }
          }
        }
      },
      "type": "function"
    },
    {
      "function": {
        "name": "my_cat_born_date",
        "description": "returns my cat's born date",
        "parameters": {
          "type": "object",
          "properti

In [6]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_YVL9vs0toLp8ynI9AJZFlTrd', created_at=1710871163, metadata={}, object='thread')


In [7]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=QUESTION
)

thread_messages = client.beta.threads.messages.list(thread.id)

print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_x090sUxs5kvogjLH257cIP3I",
      "assistant_id": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "How many flights do we have between my cat born date and Easter 2021?"
          },
          "type": "text"
        }
      ],
      "created_at": 1710871227,
      "file_ids": [],
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_YVL9vs0toLp8ynI9AJZFlTrd"
    }
  ],
  "object": "list",
  "first_id": "msg_x090sUxs5kvogjLH257cIP3I",
  "last_id": "msg_x090sUxs5kvogjLH257cIP3I",
  "has_more": false
}


In [8]:
import time, json

run            = client.beta.threads.runs.create(
  thread_id    = thread.id,
  assistant_id = assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

print(f"Run status: {run.status}")

Run status: queued


In [12]:
print(f"Run status: {run.status}")

Run status: queued


In [24]:
# Retrieve Updated Run Status

run_json = json.loads(client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).model_dump_json())
run_json

{'id': 'run_ICHPakdfrhrphgDE0lXd5eDs',
 'assistant_id': 'asst_RN5yZ4Kk1nzxC8Sx6EskFfxs',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1710871267,
 'expires_at': 1710871867,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a helpful AI assistant who helps answering questions',
 'last_error': None,
 'metadata': {},
 'model': 'gpt4-1106-128k',
 'object': 'thread.run',
 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_kmmNhnZDyJ3Iex0HW8rHAQQ7',
     'function': {'arguments': '{"date_1": "2014-03-19", "date_2": "2021-04-04"}',
      'name': 'get_flights'},
     'type': 'function'}]},
  'type': 'submit_tool_outputs'},
 'started_at': 1710871816,
 'status': 'requires_action',
 'thread_id': 'thread_YVL9vs0toLp8ynI9AJZFlTrd',
 'tools': [{'function': {'name': 'get_flights',
    'description': 'returns the number of flights between two dates',
    'parameters': {'type': 'object',
     'properties': {'date_1': {'type': 'string',
       'descripti

In [25]:
tc_id = run_json["required_action"]["submit_tool_outputs"]["tool_calls"][0]["id"]
tc_id

'call_kmmNhnZDyJ3Iex0HW8rHAQQ7'

In [26]:
tc_function_name = run_json["required_action"]["submit_tool_outputs"]["tool_calls"][0]["function"]["name"]
tc_function_name

'get_flights'

In [27]:
tc_args = run_json["required_action"]["submit_tool_outputs"]["tool_calls"][0]["function"]["arguments"]
tc_args

'{"date_1": "2014-03-19", "date_2": "2021-04-04"}'

In [28]:
import json
tool_response = eval(tc_function_name)(**json.loads(tc_args))
tool_response

'{"flights": 2573}'

In [29]:
tool_response_full = {"tool_call_id": tc_id, "output":tool_response}
tool_response_full

{'tool_call_id': 'call_kmmNhnZDyJ3Iex0HW8rHAQQ7',
 'output': '{"flights": 2573}'}

In [30]:
tool_responses = []

tool_responses.append(tool_response_full)

tool_responses

[{'tool_call_id': 'call_kmmNhnZDyJ3Iex0HW8rHAQQ7',
  'output': '{"flights": 2573}'}]

In [31]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    tool_outputs=tool_responses,
    run_id=run.id
)

run.status

BadRequestError: Error code: 400 - {'error': {'message': 'Runs in status "expired" do not accept tool outputs.', 'type': 'invalid_request_error', 'param': None, 'code': None}}

# Final implementation

In [36]:
import time, json

print (f"Question: <{QUESTION}>")

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=QUESTION
)

run            = client.beta.threads.runs.create(
  thread_id    = thread.id,
  assistant_id = assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

start_time = time.time()
status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print( f"Run status: {status} after {int((time.time() - start_time) // 60)} minutes {int((time.time() - start_time) % 60)} seconds")
    
    if status == "requires_action":
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_responses = []
        for tc in tool_calls:
            function_to_call = tc.function.name
            function_args = tc.function.arguments
            tool_response = eval(function_to_call)(**json.loads(function_args))
            tool_responses.append({"tool_call_id": tc.id, "output": tool_response})

            print(f"Calling {tc.function.name}(**{tc.function.arguments}) --> {tool_response}")

        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id, run_id=run.id, tool_outputs=tool_responses)

    time.sleep(5)
    status = run.status
        
print( f"\nRun status: {status} after {int((time.time() - start_time) // 60)} minutes {int((time.time() - start_time) % 60)} seconds")
messages = client.beta.threads.messages.list(thread_id=thread.id)
print(json.loads(messages.json())["data"][0]["content"][0]["text"]["value"])

Question: <How many flights do we have between my cat born date and Easter 2021?>
Run status: queued after 0 minutes 0 seconds
Run status: queued after 0 minutes 5 seconds
Run status: requires_action after 0 minutes 10 seconds
Calling my_cat_born_date(**{}) --> {"cat_born_date": "2014-03-19"}
Run status: queued after 0 minutes 16 seconds
Run status: in_progress after 0 minutes 21 seconds
Run status: in_progress after 0 minutes 26 seconds
Run status: in_progress after 0 minutes 32 seconds
Run status: requires_action after 0 minutes 37 seconds
Calling get_flights(**{"date_1":"2014-03-19","date_2":"2021-04-04"}) --> {"flights": 2573}
Run status: queued after 0 minutes 43 seconds

Run status: completed after 0 minutes 48 seconds
Between your cat's birthdate on March 19, 2014, and Easter Sunday on April 4, 2021, there have been a total of 2,573 flights.
